# Notas de la tutoría

https://docs.google.com/document/d/1OFimhBpQRmyhBzsIPMzMCz5L3U5bnUaSXxvbL6H2dS0/edit?usp=sharing

# Importar librerías

In [ ]:
from google.colab import drive
drivedir = r'/content/drive'
drive.mount(drivedir)

import os
import numpy as np
import pandas as pd
import datetime as dt
!pip install category_encoders

# Visualización de datos
import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns


# Configuración
pd.pandas.set_option('display.max_columns', None)
rootdir = os.path.join(drivedir, '/content/drive/MyDrive/Colab Notebooks/Store Sales')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Cargar datos

In [ ]:
data = pd.read_csv(os.path.join(rootdir, 'train.csv'))
test = pd.read_csv(os.path.join(rootdir, 'test.csv'))

transactions=pd.read_csv(os.path.join(rootdir, 'transactions.csv'))
stores=pd.read_csv(os.path.join(rootdir, 'stores.csv'))
oil=pd.read_csv(os.path.join(rootdir, 'oil.csv'))
holidays_events=pd.read_csv(os.path.join(rootdir, 'holidays_events.csv'))

# Procesar datos

## Establecer tipos de datos correctos

In [ ]:
data_processed = data.copy()
test_processed = test.copy()
transaction_processed = transactions.copy()
stores_processed = stores.copy()
oil_processed = oil.copy()
holidays_events_processed = holidays_events.copy()

# Establecemos los tipos de datos correctos
data_processed['date'] = data_processed['date'].astype('datetime64[ns]')
test_processed['date'] = test_processed['date'].astype('datetime64[ns]')
oil_processed['date'] = pd.to_datetime(oil_processed['date'])
holidays_events_processed['date'] = pd.to_datetime(holidays_events_processed['date'])
transaction_processed['date'] = pd.to_datetime(transaction_processed['date'])

oil_processed

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20
...,...,...
1213,2017-08-25,47.65
1214,2017-08-28,46.40
1215,2017-08-29,46.46
1216,2017-08-30,45.96


## Interpolar precios del petróleo

Hay valores nulos y también faltan filas para los fines de semana, que también las vamos a rellenar porque el fin de semana las tiendas tienen datos de ventas.

Primero añadimos las filas que faltan y luego se interpola.

In [ ]:
if 'date' in oil_processed.columns:
  oil_processed.set_index('date', inplace=True)
oil_processed.index = pd.to_datetime(oil_processed.index)


new_date_range = pd.date_range(start=oil_processed.index[0], end=oil_processed.index[-1], freq="D")
oil_processed = oil_processed.reindex(new_date_range)
oil_processed.index.name = 'date'

# Usamos el método "time" en la interpolación ya que solo hay datos de lunes a viernes, así que no son continuos del todo
# Usamos limit_direction="both" para que también interpole el primer valor de la serie, que está vacío

oil_processed.interpolate(method='time', limit_direction='both', inplace=True)
oil_processed

DatetimeIndex([], dtype='datetime64[ns]', freq='D')

In [ ]:
oil_processed.to_csv( os.path.join( rootdir, 'oil_processed.csv'), index=True, encoding='utf-8' )

# Juntar datasets

## Añadir datos de cada tienda

Existe un dataset de tiendas, el cual tiene los datos de cada tienda, según el valor store_nbr.

Los dataset train y test tienen una columna llamada store_nbr.

Vamos ainsertar los datos de cada tienda en los dataset train y test.

In [ ]:
data_processed = pd.merge(data_processed, stores_processed, on="store_nbr", how="left")
data_processed.head()

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
1,1,2013-01-01,1,BABY CARE,0.0,0,Quito,Pichincha,D,13
2,2,2013-01-01,1,BEAUTY,0.0,0,Quito,Pichincha,D,13
3,3,2013-01-01,1,BEVERAGES,0.0,0,Quito,Pichincha,D,13
4,4,2013-01-01,1,BOOKS,0.0,0,Quito,Pichincha,D,13


In [ ]:
test_processed = pd.merge (test_processed, stores_processed, on="store_nbr", how="left")
test_processed.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13


## Añadir precio del petróleo

In [ ]:
#Vamos a incorporar ahora el valor del oil en los dos dataset

if 'dcoilwtico' not in data_processed.columns:
  data_processed = pd.merge(data_processed, oil_processed, on="date", how="left")
data_processed.head()

if 'dcoilwtico' not in test_processed.columns:
  test_processed = pd.merge(test_processed, oil_processed, on="date", how="left")
test_processed.head()

,id,date,store_nbr,family,onpromotion,city,state,type,cluster,dcoilwtico
0,3000888,2017-08-16,1,AUTOMOTIVE,0,Quito,Pichincha,D,13,46.8
1,3000889,2017-08-16,1,BABY CARE,0,Quito,Pichincha,D,13,46.8
2,3000890,2017-08-16,1,BEAUTY,2,Quito,Pichincha,D,13,46.8
3,3000891,2017-08-16,1,BEVERAGES,20,Quito,Pichincha,D,13,46.8
4,3000892,2017-08-16,1,BOOKS,0,Quito,Pichincha,D,13,46.8


## Añadir terremoto

In [ ]:
data_processed['event_earthquake'] = 0
test_processed['event_earthquake'] = 0
data_processed.loc[data_processed['date'] == '2016-04-16 00:00:00', 'event_earthquake'] = 1

## Quitar ID

El ID puede generar ruido

In [ ]:
data_processed.drop('id', axis=1, inplace=True)
test_processed.drop('id', axis=1, inplace=True)

# Guardar el dataset combinado

Guardamos el fichero para poder trabajar con él en otro momento sin tener que procesar todo de nuevo.

In [ ]:
# Guardamos el dataset
data_processed.to_csv( os.path.join( rootdir, 'data_processed_2.csv'), index=False, encoding='utf-8' )
test_processed.to_csv( os.path.join( rootdir, 'test_processed_2.csv'), index=False, encoding='utf-8' )

## Cargar el dataset combinado

In [ ]:
data_processed = pd.read_csv(os.path.join(rootdir, 'data_processed_2.csv'))
test_processed = pd.read_csv(os.path.join(rootdir, 'test_processed_2.csv'))

In [ ]:
data_processed

,id,date,store_nbr,family,sales,onpromotion,month,day,day_of_week,sales_media_movil_30_dias_lag16,city,state,cluster,dcoilwtico,holiday_locale,holiday_type,event_earthquake,family_name,type_A,type_B,type_C,type_D,type_E
0,0,2013-01-01,1,6.518422,0.000,0,1,1,1,0.000000,565.589351,562.317708,13,93.14,1,1,0,AUTOMOTIVE,0,0,0,1,0
1,1,2013-01-01,1,0.118086,0.000,0,1,1,1,4.666667,565.589351,562.317708,13,93.14,1,1,0,BABY CARE,0,0,0,1,0
2,2,2013-01-01,1,3.969794,0.000,0,1,1,1,0.000000,565.589351,562.317708,13,93.14,1,1,0,BEAUTY,0,0,0,1,0
3,3,2013-01-01,1,2548.927182,0.000,0,1,1,1,3.700000,565.589351,562.317708,13,93.14,1,1,0,BEVERAGES,0,0,0,1,0
4,4,2013-01-01,1,0.075638,0.000,0,1,1,1,2227.433333,565.589351,562.317708,13,93.14,1,1,0,BOOKS,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2780311,3000883,2017-08-15,9,374.500734,438.133,0,8,15,1,2.633333,565.589351,562.317708,6,47.57,0,0,0,POULTRY,0,1,0,0,0
2780312,3000884,2017-08-15,9,103.387085,154.553,1,8,15,1,7.633333,565.589351,562.317708,6,47.57,0,0,0,PREPARED FOODS,0,1,0,0,0
2780313,3000885,2017-08-15,9,1441.617142,2419.729,148,8,15,1,596.142265,565.589351,562.317708,6,47.57,0,0,0,PRODUCE,0,1,0,0,0
2780314,3000886,2017-08-15,9,3.164105,121.000,8,8,15,1,77.572233,565.589351,562.317708,6,47.57,0,0,0,SCHOOL AND OFFICE SUPPLIES,0,1,0,0,0


In [ ]:
test_processed

,id,date,store_nbr,family,sales,onpromotion,month,day,day_of_week,sales_media_movil_30_dias_lag16,city,state,cluster,dcoilwtico,holiday_locale,holiday_type,event_earthquake,family_name,type_A,type_B,type_C,type_D,type_E
0,3000888,2017-08-16,1,6.518422,0.0,0,8,16,2,1.100000,576.982379,573.969128,13,46.80,0,0,0,AUTOMOTIVE,0,0,0,1,0
1,3000889,2017-08-16,1,0.151202,0.0,0,8,16,2,0.666667,576.982379,573.969128,13,46.80,0,0,0,BABY CARE,0,0,0,1,0
2,3000890,2017-08-16,1,3.969794,0.0,2,8,16,2,0.933333,576.982379,573.969128,13,46.80,0,0,0,BEAUTY,0,0,0,1,0
3,3000891,2017-08-16,1,2548.927182,0.0,20,8,16,2,1.066667,576.982379,573.969128,13,46.80,0,0,0,BEVERAGES,0,0,0,1,0
4,3000892,2017-08-16,1,0.149111,0.0,0,8,16,2,0.500000,576.982379,573.969128,13,46.80,0,0,0,BOOKS,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28507,3029395,2017-08-31,9,374.500734,0.0,1,8,31,3,7.600000,576.982379,573.969128,6,47.26,0,0,0,POULTRY,0,1,0,0,0
28508,3029396,2017-08-31,9,103.387085,0.0,0,8,31,3,0.000000,576.982379,573.969128,6,47.26,0,0,0,PREPARED FOODS,0,1,0,0,0
28509,3029397,2017-08-31,9,1441.617142,0.0,1,8,31,3,0.000000,576.982379,573.969128,6,47.26,0,0,0,PRODUCE,0,1,0,0,0
28510,3029398,2017-08-31,9,3.164105,0.0,9,8,31,3,0.000000,576.982379,573.969128,6,47.26,0,0,0,SCHOOL AND OFFICE SUPPLIES,0,1,0,0,0
